The SHM model was calibrated using 3 different methods. In this notebook we select the best calibration set for each basin, and get the variables of interest for each case

In [15]:
#Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from hydrological_models import SHM
from spotpy_calibration import read_information
from spotpy_calibration import NSE_loss

In [3]:
# Initialize information
path_entities= '../data/basin_id/basins_camels_gb_60.txt'
path_ts = '../data/CAMELS_GB/timeseries/'
warmup = 365
forcing_variables = ['precipitation', 'peti', 'temperature']
target_variables = ['discharge_spec']
testing_period = ['1997-01-01','2008-12-31']

# Save results
path_output = '../results/models/conceptual_models/'
selected_basins_id= np.loadtxt(path_entities, dtype='str').tolist()

In [9]:
# Read the calibration results by each method, and select the best case. In other words, select the calibrated
# parameters (for each basin) that gave best results. The SHM_XXX_summary.csv files are generated when running the 
# scripts associated with each calibration method

# Read DREAM calibration
df_dream = pd.read_csv(path_output+'SHM_dream_summary.csv', dtype={'basin_id': str})
df_dream.set_index('basin_id', inplace=True)

# Read SCE calibration
df_sce = pd.read_csv(path_output+'SHM_sce_summary.csv', dtype={'basin_id': str})
df_sce.set_index('basin_id', inplace=True)

# Read rope calibration
df_rope = pd.read_csv(path_output+'SHM_rope_summary.csv', dtype={'basin_id': str})
df_rope.set_index('basin_id', inplace=True)

# The last column of each dataset is the NSE in testing.
last_column_values = pd.concat([df_dream.iloc[:, -1], df_sce.iloc[:, -1], df_rope.iloc[:, -1]], axis=1,  
                               keys=['dream', 'sce', 'rope'])

max_value_index = last_column_values.idxmax(axis=1)

# Select the best parameter set for each basin
parameter_sets = pd.concat([df_dream[max_value_index=='dream'].iloc[:, 1:-1],
                            df_sce[max_value_index=='sce'].iloc[:, 1:-1],
                            df_rope[max_value_index=='rope'].iloc[:, 1:-1]], axis=0)

parameter_sets= parameter_sets.reindex(selected_basins_id)

Run the model for each basin, using the best calibration parameters

In [47]:
hyd_model = SHM()
test_results = {}
NSE_testing =  []

time_index= pd.date_range(start=testing_period[0], end=testing_period[1])
# Loop that goes through each basin
for i, basin in enumerate(selected_basins_id):
     
     input, target = read_information(path_ts = path_ts, 
                                     basin_id = basin, 
                                     forcing = forcing_variables, 
                                     target = target_variables, 
                                     time_period = testing_period)
     
     
     out, states = hyd_model.run_model(input=input, param=parameter_sets.loc[basin].values)
     
     
     
     df_discharge = pd.DataFrame(data={'discharge': out[warmup:,:].flatten()}, index=time_index[warmup:])
     
     internal_states = {key: value[warmup:, :].flatten() for key, value in states.items()}
     df_buckets = pd.DataFrame(data=internal_states, index=time_index[warmup:]) 


     test_results[basin] = {'discharges': df_discharge,
                            'internal_states': df_buckets}
       
     NSE_testing.append(NSE_loss(evaluation=target[warmup:][~np.isnan(target)[warmup:,0]].flatten(),
                                 simulation=out[warmup:][~np.isnan(target)[warmup:,0]].flatten()))

Final NSE for testing period for all basins

In [60]:
df_NSE = pd.DataFrame(data={'basin_id': selected_basins_id,'NSE': NSE_testing})
df_NSE = df_NSE.set_index('basin_id')
df_NSE.to_csv(path_output+'/SHM_NSE.csv', index=True, header=True)

Generate soil moisture series of SHM model

In [56]:
# Generate soil moisture series
bucket_name = 'su'
buckets =  {key: value['internal_states'][bucket_name] for key, value in test_results.items()}
df_buckets = pd.DataFrame(data=buckets)
df_buckets.index.name = 'date'
df_buckets.to_csv(path_output+'/SHM_'+bucket_name+'.csv', index=True, header=True)